In [1]:
import datetime

start = datetime.datetime.now()
start

datetime.datetime(2021, 11, 5, 22, 39, 21, 818790)

In [2]:
import sqlite3
import pandas as pd
import networkx as nx
from collections import defaultdict
from tqdm.auto import tqdm
import math
from itertools import combinations
import pickle

db = sqlite3.connect("datasets.db")
cur = db.cursor()

In [3]:
book_history = pd.read_sql_query("""
SELECT 
    circulation_short.readerID, 
    group_concat(books.rubric1) as rubrics, 
    group_concat(books.author_id) as authors
FROM circulation_short
JOIN books ON circulation_short.smart = books.smart_collapse_field
WHERE rubric1 IS NOT NULL AND author_id IS NOT NULL
GROUP BY readerID
HAVING count(smart) > 1
""", con=db)
book_history.shape

(304096, 3)

In [4]:
book_history["rubrics"] = book_history["rubrics"].apply(lambda x: [int(i) for i in x.split(",")])
book_history["authors"] = book_history["authors"].apply(lambda x: [int(i) for i in x.split(",")])

In [5]:
book_history.head(3)

,readerID,rubrics,authors
0,163,"[252, 479]","[91143, 130512]"
1,170,"[610, 610]","[121213, 27741]"
2,171,"[479, 479, 479, 479, 479, 479, 479, 479, 479, 18]","[47875, 132224, 134206, 104699, 94970, 141471,..."


In [6]:
edges = defaultdict(lambda: defaultdict(int))
count_rubric = defaultdict(lambda: defaultdict(int))
count_name = defaultdict(int)

for row in tqdm(book_history.values):
    row_dict = defaultdict(list)
    
    for r, a in zip(row[1], row[2]):
        row_dict[r].append(a)
        count_rubric[a][r] += 1
        count_name[a] += 1
    
    for r in row_dict:
        for o in combinations(row_dict[r], 2):
            edges[o][r] += 1

  0%|          | 0/304096 [00:00<?, ?it/s]

In [7]:
del book_history

In [8]:
edges2 = defaultdict(lambda: defaultdict(int))
pairs = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for pair, links in tqdm(edges.items()):
    if sum(links.values()) >= 3:
        for rubric, val in links.items():
            if val > 1:
                coef = 14 - math.log(val / (count_rubric[pair[0]][rubric] + count_rubric[pair[1]][rubric]))
                edges2[rubric][pair] = coef
                pairs[rubric][pair[0]][pair[1]] = coef
                pairs[rubric][pair[1]][pair[0]] = coef

pairs2 = defaultdict(lambda: defaultdict(set))
for r in pairs:
    for i in pairs[r]:
        pairs2[r][i] = set([i for i in sorted(pairs[r][i], key=pairs[r][i].get)][:10])

  0%|          | 0/9086950 [00:00<?, ?it/s]

In [9]:
graphs = defaultdict(lambda: nx.Graph())

for pair, links in tqdm(edges.items()):
    if sum(links.values()) >= 3:
        for rubric, val in links.items():
            if val > 1:
                if pair[0] in pairs2[rubric][pair[1]] or pair[1] in pairs2[rubric][pair[0]]:
                    graphs[rubric].add_edge(*pair, weight=edges2[rubric][pair])

  0%|          | 0/9086950 [00:00<?, ?it/s]

In [10]:
count_rubric.get(23203)

defaultdict(int, {479: 2808, 534: 470, 218: 97, 541: 49})

In [11]:
view = graphs[479][23203]
sorted(view.items(), key=lambda x: x[1]["weight"])[:20]

[(23203, {'weight': 17.64294235892691}),
 (45648, {'weight': 17.818819132932095}),
 (15583, {'weight': 18.159705113821087}),
 (64459, {'weight': 18.201963395525198}),
 (113922, {'weight': 18.251233193179736}),
 (130211, {'weight': 18.337442728065856}),
 (50649, {'weight': 18.351801701525492}),
 (134228, {'weight': 18.37260741275739}),
 (85642, {'weight': 18.426895225190133}),
 (134391, {'weight': 18.467735079883095}),
 (120840, {'weight': 18.516807246698836}),
 (96072, {'weight': 20.150247339261803}),
 (143271, {'weight': 20.156978985585553}),
 (133968, {'weight': 20.55677835615804}),
 (35601, {'weight': 20.557488331338735}),
 (147728, {'weight': 20.56526497003536}),
 (14821, {'weight': 20.841971538436134}),
 (97614, {'weight': 20.842683282238422}),
 (31037, {'weight': 20.842683282238422}),
 (105366, {'weight': 20.843038964262448})]

In [12]:
with open("author_graphs.pkl", "wb") as f:
    pickle.dump(dict(graphs), f)

In [13]:
print(datetime.datetime.now())
print(datetime.datetime.now() - start)

2021-11-05 22:40:24.491768
0:01:02.673399
